In [1]:
import os

In [2]:
%pwd

'/Users/reetu/Documents/Personal_Projects/chicken_disease_classification/Chicken-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/reetu/Documents/Personal_Projects/chicken_disease_classification/Chicken-Disease-Classification'

In [103]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float  # Add this field
    params_classes: int          # Add this field
    params_include_top: bool     # Add this field
    params_weights: str

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [104]:
import sys
import os

# Clear out any previous paths in the notebook by restarting kernel

# Append the correct path to the 'src' directory inside the nested folder
sys.path.append('/Users/reetu/Documents/Personal_Projects/chicken_disease_classification/Chicken-Disease-Classification/src')

# Print the final sys.path to verify the path is added only once
# print("Final sys.path:", sys.path)

# Try importing the modules again
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [105]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params['EPOCHS'],
        params_batch_size=params['BATCH_SIZE'],
        params_is_augmentation=params['AUGMENTATION'],
        params_image_size=tuple(params['IMAGE_SIZE']),  # Make sure IMAGE_SIZE is passed as a tuple
        params_learning_rate=params['LEARNING_RATE'],
        params_classes=params['CLASSES'],
        params_include_top=params['INCLUDE_TOP'],
        params_weights=params['WEIGHTS']
        )

        return training_config

In [106]:
import time

In [107]:
import tensorflow as tf
import os
import time
from pathlib import Path

class PrepareCallback:
    def __init__(self, config):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            str(self.config.tensorboard_root_log_dir),  # Convert PosixPath to string
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),  # Convert PosixPath to string
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [108]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [109]:
# from tensorflow.keras.layers import InputLayer

# class Training:
#     def __init__(self, config: TrainingConfig):
#         self.config = config
    
#     def get_base_model(self):
#         # Load the model, handling InputLayer deserialization
#         self.model = tf.keras.models.load_model(
#             self.config.updated_base_model_path,
#             custom_objects={'InputLayer': InputLayer}  # Handle the InputLayer issue
#         )
    
#     def train_valid_generator(self):

#         datagenerator_kwargs = dict(
#             rescale = 1./255,
#             validation_split=0.20
#         )

#         dataflow_kwargs = dict(
#             target_size=self.config.params_image_size[:-1],
#             batch_size=self.config.params_batch_size,
#             interpolation="bilinear"
#         )

#         valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
#             **datagenerator_kwargs
#         )

#         self.valid_generator = valid_datagenerator.flow_from_directory(
#             directory=self.config.training_data,
#             subset="validation",
#             shuffle=False,
#             **dataflow_kwargs
#         )

#         if self.config.params_is_augmentation:
#             train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
#                 rotation_range=40,
#                 horizontal_flip=True,
#                 width_shift_range=0.2,
#                 height_shift_range=0.2,
#                 shear_range=0.2,
#                 zoom_range=0.2,
#                 **datagenerator_kwargs
#             )
#         else:
#             train_datagenerator = valid_datagenerator

#         self.train_generator = train_datagenerator.flow_from_directory(
#             directory=self.config.training_data,
#             subset="training",
#             shuffle=True,
#             **dataflow_kwargs
#         )

#     @staticmethod
#     def save_model(path: Path, model: tf.keras.Model):
#         model.save(path)


#     def train(self, callback_list: list):
#         self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
#         self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

#         self.model.fit(
#             self.train_generator,
#             epochs=self.config.params_epochs,
#             steps_per_epoch=self.steps_per_epoch,
#             validation_steps=self.validation_steps,
#             validation_data=self.valid_generator,
#             callbacks=callback_list
#         )

#         self.save_model(
#             path=self.config.trained_model_path,
#             model=self.model
#         )

In [110]:
# """This works perfectly fine"""

# from tensorflow.keras.layers import InputLayer

# class Training:
#     def __init__(self, config: TrainingConfig):
#         self.config = config
    
#     def get_base_model(self):
#         # Instead of loading an old model with batch_shape, we rebuild it using input_shape
#         self.model = tf.keras.applications.vgg16.VGG16(
#             input_shape=self.config.params_image_size,  # Use input_shape to avoid the batch_shape issue
#             weights=self.config.params_weights,
#             include_top=self.config.params_include_top
#         )

#         # Add custom layers if needed (e.g., for classification tasks)
#         flatten_in = tf.keras.layers.Flatten()(self.model.output)
#         prediction = tf.keras.layers.Dense(
#             units=self.config.params_classes,
#             activation="softmax"
#         )(flatten_in)

#         # Build the full model
#         self.model = tf.keras.models.Model(inputs=self.model.input, outputs=prediction)

#         # Compile the model
#         self.model.compile(
#             optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
#             loss='categorical_crossentropy',
#             metrics=['accuracy']
#         )

#         # Save the newly built model
#         self.save_model(path=self.config.updated_base_model_path, model=self.model)
    
#     def train_valid_generator(self):

#         datagenerator_kwargs = dict(
#             rescale=1./255,
#             validation_split=0.20
#         )

#         dataflow_kwargs = dict(
#             target_size=self.config.params_image_size[:-1],
#             batch_size=self.config.params_batch_size,
#             interpolation="bilinear"
#         )

#         valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
#             **datagenerator_kwargs
#         )

#         self.valid_generator = valid_datagenerator.flow_from_directory(
#             directory=self.config.training_data,
#             subset="validation",
#             shuffle=False,
#             **dataflow_kwargs
#         )

#         if self.config.params_is_augmentation:
#             train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
#                 rotation_range=40,
#                 horizontal_flip=True,
#                 width_shift_range=0.2,
#                 height_shift_range=0.2,
#                 shear_range=0.2,
#                 zoom_range=0.2,
#                 **datagenerator_kwargs
#             )
#         else:
#             train_datagenerator = valid_datagenerator

#         self.train_generator = train_datagenerator.flow_from_directory(
#             directory=self.config.training_data,
#             subset="training",
#             shuffle=True,
#             **dataflow_kwargs
#         )

#     @staticmethod
#     def save_model(path: Path, model: tf.keras.Model):
#         model.save(path)

#     def train(self, callback_list: list):
#         self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
#         self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

#         self.model.fit(
#             self.train_generator,
#             epochs=self.config.params_epochs,
#             steps_per_epoch=self.steps_per_epoch,
#             validation_steps=self.validation_steps,
#             validation_data=self.valid_generator,
#             callbacks=callback_list
#         )

#         self.save_model(
#             path=self.config.trained_model_path,
#             model=self.model
#         )


In [121]:
from PIL import Image
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        # Instead of loading an old model with batch_shape, we rebuild it using input_shape
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,  # Use input_shape to avoid the batch_shape issue
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        # Add custom layers if needed (e.g., for classification tasks)
        flatten_in = tf.keras.layers.Flatten()(self.model.output)
        prediction = tf.keras.layers.Dense(
            units=self.config.params_classes,
            activation="softmax"
        )(flatten_in)

        # Build the full model
        self.model = tf.keras.models.Model(inputs=self.model.input, outputs=prediction)

        # Compile the model
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Save the newly built model
        self.save_model(path=self.config.updated_base_model_path, model=self.model)

    def manual_image_loader(self, image_dir, label_map):
        images = []
        labels = []
        
        for label, idx in label_map.items():
            class_dir = os.path.join(image_dir, label)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                try:
                    img = Image.open(img_path)
                    img = img.resize(self.config.params_image_size[:-1])  # Resize image to match model input
                    img = img_to_array(img)
                    images.append(img)
                    labels.append(idx)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
                    continue

        images = np.array(images) / 255.0  # Normalize the pixel values between 0 and 1
        labels = to_categorical(labels, num_classes=self.config.params_classes)
        
        return images, labels

    def train_valid_generator(self):
        # Define a label map for your dataset (e.g., {"healthy": 0, "coccidiosis": 1})
        label_map = {"healthy": 0, "coccidiosis": 1}

        # Manually load the images and labels
        all_images, all_labels = self.manual_image_loader(self.config.training_data, label_map)

        # Split into training and validation sets
        self.train_images, self.valid_images, self.train_labels, self.valid_labels = train_test_split(
            all_images, all_labels, test_size=0.20, random_state=42)

    def save_model(self, path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        # Fit the model manually using the loaded data
        self.model.fit(
            self.train_images, self.train_labels,
            validation_data=(self.valid_images, self.valid_labels),
            epochs=self.config.params_epochs,
            batch_size=self.config.params_batch_size,
            callbacks=callback_list
        )

        # Save the model after training
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [120]:

!pip install scikit-learn


  Using cached scikit_learn-1.3.2-cp38-cp38-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.10.1-cp38-cp38-macosx_12_0_arm64.whl.metadata (53 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.3.2-cp38-cp38-macosx_12_0_arm64.whl (9.4 MB)
Using cached scipy-1.10.1-cp38-cp38-macosx_12_0_arm64.whl (28.8 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [116]:
from PIL import Image
print("Pillow is successfully installed and available.")


Pillow is successfully installed and available.


In [122]:

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-09-13 21:25:32,599: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-13 21:25:32,601: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-13 21:25:32,602: INFO: common: created directory at: artifacts]
[2024-09-13 21:25:32,602: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-09-13 21:25:32,602: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2024-09-13 21:25:32,604: INFO: common: created directory at: artifacts/training]
[2024-09-13 21:25:32,862: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.]
[2024-09-13 21:25:32,863: WARNING: __init__: There is a known slowdown when using v2.11+ Keras optimizers on M1/M2 Macs. Falling back to the legacy Keras optimizer, i.e., `tf.keras.optimizers.legacy.Adam`.]


/opt/miniconda3/envs/chicken/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 87s 4s/step - loss: 1306740224.0000 - accuracy: 0.5288 - val_loss: 2624.5217 - val_accuracy: 0.4103
